搭建的神经网络，使用olivettiface数据集进行训练

In [89]:
#导入必要的包
import torch
import torch.nn as nn                           #模型工具包
import torch.optim as optim                     #优化器
from torchvision.transforms.v2 import ToTensor  #转换图像为张量
from sklearn.datasets import fetch_olivetti_faces     #数据集
from sklearn.model_selection import train_test_split  #数据拆分
from torch.utils.data import DataLoader, TensorDataset         #数据加载器 ，优化大量数据

In [104]:
#定义超参数
LR = 1e-3
epochs = 20
BATCH_SIZE = 32 #数据批次拆分

In [105]:
#数据加载
face_data = fetch_olivetti_faces(data_home = './data',shuffle=True, random_state=42)

In [106]:
#数据拆分
X = torch.tensor(face_data.images.reshape(400,-1))
y = torch.tensor(face_data.target, dtype=torch.long)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [107]:
#数据集创建
face_train_data = TensorDataset(X_train, y_train)
face_test_data = TensorDataset(X_test, y_test)

In [108]:
#数据加载器
train_loader = DataLoader(face_train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(face_test_data, batch_size=BATCH_SIZE, shuffle=False)   

In [109]:
#形状确认
face_train_data[1][0].reshape(-1).shape #将图像转为1维向量

torch.Size([4096])

In [110]:
#模型调用
from torch_week4_model import TorcFace
model = TorcFace()

In [111]:
##5.损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [114]:
#6.模型的训练
for epoch in range(epochs):
    #提取训练的数据  数据导入到模型中
    for data,target in train_loader:
        out_put = model(data)           #前向运算/前向传播
        loss = loss_fn(out_put,target)  #计算损失
        #反向传播 
        optimizer.zero_grad()           #所有参数梯度清零
        loss.backward()                 #计算梯度(参数，grad)
        optimizer.step()                #更新参数
        print(f'Epoch:{epoch},Loss:{loss.item()}')

Epoch:0,Loss:0.19826005399227142
Epoch:0,Loss:0.11052684485912323
Epoch:0,Loss:0.14646679162979126
Epoch:0,Loss:0.1495869904756546
Epoch:0,Loss:0.09262598305940628
Epoch:0,Loss:0.1379576474428177
Epoch:0,Loss:0.11275103688240051
Epoch:0,Loss:0.17236575484275818
Epoch:0,Loss:0.1532813459634781
Epoch:0,Loss:0.13539882004261017
Epoch:1,Loss:0.11484779417514801
Epoch:1,Loss:0.09896522015333176
Epoch:1,Loss:0.10863310098648071
Epoch:1,Loss:0.16744859516620636
Epoch:1,Loss:0.1220143511891365
Epoch:1,Loss:0.08022427558898926
Epoch:1,Loss:0.1624944657087326
Epoch:1,Loss:0.11694033443927765
Epoch:1,Loss:0.10143576562404633
Epoch:1,Loss:0.14593233168125153
Epoch:2,Loss:0.11930884420871735
Epoch:2,Loss:0.13153311610221863
Epoch:2,Loss:0.12201989442110062
Epoch:2,Loss:0.09756243973970413
Epoch:2,Loss:0.11900097876787186
Epoch:2,Loss:0.06256412714719772
Epoch:2,Loss:0.08191322535276413
Epoch:2,Loss:0.1335870921611786
Epoch:2,Loss:0.09735909104347229
Epoch:2,Loss:0.18964339792728424
Epoch:3,Loss:0.1

In [115]:
##6.0测试
test_dl = DataLoader(test_loader,batch_size= BATCH_SIZE)
correct = 0
total = 0
with torch.no_grad():    #不进行梯度计算
    for data,target in test_loader:
        output = model(data) 
        _,predicted = torch.max(output,1)
        total += target.size(0) #size(0) = shape[0]
        correct+= (predicted == target).sum().item()
print(f'{correct/total*100}%')   

93.0%
